# Séquences d'événements logiciels

Jusqu'ici j'ai analysé les données brutes sans faire de traitements qui n'étaient pas nécessaires à l'affichage. Mais les données représentent des événements logiciels comme naviguer vers une page, charger une vidéo, quitter une page et non pas des événements directement liés à l'apprentissage.

Afin de pouvoir montrer des données pertinentes aux utilisateurs, il faudrait déterminer ce à quoi correspondent certaines séquences d'événements logiciels et lier ces séquences à un objectif éducatif.




## Obtenir la liste d'événements
Dans un premier temps, il faut lister les actions effectuées et les trier par date afin d'obtenir des séquences chronologiques.

Pour cela on importe tout d'abord les données. Celles-ci étant très volumineuses car en très grande quantité, elles ont été limitées à un extrait correspondant au 40 000 premières lignes.

In [2]:
import pandas as pd
import datetime as dt

data = pd.read_csv("data/trackinglogs_course-v1_minestelecom_04026_session02_chunk_1.csv")

Les actions sont décrites dans la colone "event_type" par un URL relatif menant à différents endroits d'un cours ou par un mot clef représentant une interaction avec un élément de la page. A cela s'ajoute le contexte qui correspond à la page sur laquelle était l'utilisateur au moement de l'action.

Afin de mieux filter ces informations, on divise les infomations en jetons correspondant aux différents niveux d'imbrication des URL et on retire les identifiant communs qui correspondent au cours dont proviennent les données. Ensuite on crée une colonne "event_category" correspondant au premier niveau d'action qui correspond au type d'action effectuée.

In [3]:
data["event_type_tokens"] = data["event_type"].apply(lambda x: x.split('/'))
data["path_tokens"] = data["context.path"].apply(lambda x: x.split('/'))
for line in data["event_type_tokens"]:
    try :
        line.remove('')
        line.remove('courses')
        line.remove('course-v1:MinesTelecom+04026+session02')
        line.remove('')
    except :
        pass
for line in data["path_tokens"]:
    try :
        line.remove('')
        line.remove('courses')
        line.remove('course-v1:MinesTelecom+04026+session02')
        line.remove('')
    except :
        pass
data["event_category"] = data["event_type_tokens"].apply(lambda x: 'null' if len(x) == 0 else x[0])

Afin de mieux visualiser les informations, on fait le tri sur les catégories d'actions ainsi que sur l'horodatage de celles-ci.

In [4]:
user687522 = data[data["context.user_id"] == 1055391].sort_values(by='time')

#display value couts
#print(data["context.user_id"].value_counts())
user1055391 = data[data["context.user_id"] == 1055391].sort_values(by='time')
user617997  = data[data["context.user_id"] == 617997 ].sort_values(by='time')
user7214527 = data[data["context.user_id"] == 7214527].sort_values(by='time')
user7206957 = data[data["context.user_id"] == 7206957].sort_values(by='time')

event_category_array = pd.concat([user1055391["event_category"].reset_index(drop=True), user1055391["time"].reset_index(drop=True), user617997["event_category"].reset_index(drop=True),user617997["time"].reset_index(drop=True),user7214527["event_category"].reset_index(drop=True),user7214527["time"].reset_index(drop=True), user7206957["event_category"].reset_index(drop=True),user7206957["time"].reset_index(drop=True)],axis=1)
event_category_array.to_csv("./event_category_array.csv", index=False)


## Recherche de Séquences

Dans un premier temps, on peut chercher à repérer des séquences en observant simplement les données. Certains motifs sont facilement observables mais il pourrait être pertinent de fournir ces données à un algorithme de recherche de séquences afin de trouver des séquences que l'on ne remarque pas au premier coup d'oeil ou encore des séquences plus englobantes.
### Résolutions de problèmes

On peut observer que lors de la résolution de problèmes il y a deux événements "problem_check" quasi simultanés, un événement "xblock" correspondant à une interaction de l'utilisateur avec un élément de la page (QCM, quizz) et un événement "problem_graded".

![](./pictures/seq_problem_check_01.png)

![](./pictures/seq_problem_check_02.png)

La séquence peut commencer de deux manières différentes mais reste toujours ininterrompue sur les données étudiées et correspond à la demande d'évaluation d'un problème.

### Visionnage de videos

Lors du visionnage de vidéos, la séquences est plus compliquée car il y a beaucoup de répétitions en fonction du temps passé par l'utilisateur à visionner certains passages.

![](./pictures/seq_play_video_01.png)

Il pourrait être intéressant de comparer comment différents apprenants interagissent avec les videos.


In [5]:
data["event_category_detailed"] = data["event_type_tokens"].apply(lambda x: 'null' if len(x) == 0 else x[0] if len(x) == 1 else x[0]+'+'+str(len(x)) )

user1055391 = data[data["context.user_id"] == 1055391].sort_values(by='time')
user617997  = data[data["context.user_id"] == 617997 ].sort_values(by='time')
user7214527 = data[data["context.user_id"] == 7214527].sort_values(by='time')
user7206957 = data[data["context.user_id"] == 7206957].sort_values(by='time')

event_category_array = pd.concat([user1055391["event_category_detailed"].reset_index(drop=True), user1055391["time"].reset_index(drop=True), user617997["event_category_detailed"].reset_index(drop=True),user617997["time"].reset_index(drop=True),user7214527["event_category_detailed"].reset_index(drop=True),user7214527["time"].reset_index(drop=True), user7206957["event_category_detailed"].reset_index(drop=True),user7206957["time"].reset_index(drop=True)],axis=1)
event_category_array.to_csv("./event_category_array_detailed.csv", index=False)

parser_input = ' '.join(user1055391["event_category"].reset_index(drop=True).to_list())


##### Regrouper les evenements et mesurer le temps écoulé

Afin d'obtenir des visualisation plus pertinentes, il est important de filtrer les resultats obtenus plus tôt. En rassemblant les séquences, on peut non seulement observer un comprtement plus "humain" mais aussi déduire le temps passé sur certaines actions.


In [6]:
   
from ply import lex
import ply.yacc as yacc

tokens = (
    "WIKIACCESS",
    "COURSEWAREACCESS",
    "VIDEOSTART",
    "VIDEOINTERACT",
    "PAGENAV",
    "PROBLEMSAVE",
    "TIMESTAMP",
    "EXIT",
    "FORUM",
    "ODD"
)

t_ignore = ' \t,'

t_WIKIACCESS = r'(course_)?wiki|info'
t_COURSEWAREACCESS  = r'courseware|progress'
t_VIDEOSTART   = r'(load_)?video(_player_ready)?'
t_VIDEOINTERACT     = r'(pause|seek|play|stop)_video'
t_PAGENAV  = r'(xblock|seq_next|seq_prev|seq_goto)'
t_PROBLEMSAVE  = r'(problem_save|save_problem_success|problem_graded|showanswer|problem_check|problem_show)'
t_TIMESTAMP = r'[0-9]{4}-(0[1-9]|1[0-2])-(0[1-9]|[1-2][0-9]|3[0-1])T(2[0-3]|[01][0-9]):[0-5][0-9]:[0-5][0-9].[0-9]{6}(\+|-)(2[0-3]|[01][0-9]):[0-5][0-9]'
t_EXIT = r'page_close|null'
t_FORUM = r'edx.forum.thread.created|discussion'
t_ODD = r'58dc5075b29a4479964195d35aff3e34'
def t_error(t):
    print("Invalid Token: ", t.value)
    t.lexer.skip(1)
    
__file__ = "Sequences.ipynb" # ply wants to be run in a file not in a REPL
lexer = lex.lex()

lexer.input(parser_input)

while True:
    tok  = lexer.token()
    if not tok:
        break
    print(tok.value)
        


null
null
discussion
course_wiki
wiki
wiki
wiki
progress
courseware
xblock
courseware
page_close
courseware
page_close
xblock
seq_next
courseware
page_close
courseware
page_close
courseware
page_close
courseware
page_close
page_close
info
58dc5075b29a4479964195d35aff3e34
courseware
courseware
courseware
page_close
xblock
xblock
problem_check
problem_check
problem_graded
seq_goto
courseware
page_close
courseware
page_close
discussion
discussion
edx.forum.thread.created
courseware
page_close
courseware
page_close
courseware
page_close
courseware
page_close
xblock
xblock
seq_goto
xblock
video_player_ready
load_video
play_video
pause_video
play_video
seek_video
pause_video
seek_video
play_video
seek_video
pause_video
seek_video
play_video
pause_video
seq_goto
xblock
xblock
xblock
seq_goto
xblock
xblock
seq_goto
video_player_ready
load_video
xblock
discussion
seq_goto
seq_goto
xblock
xblock
xblock
seq_goto
xblock
seq_goto
xblock
xblock
video_player_ready
load_video
seq_goto
xblock
xblock
se

In [7]:
precedence = (
    ( 'left', 'PAGENAV')
)

def p_add( p ) :
    'expr : PAGENAV expr'
    p[0] = p[1] + [2]

parser = yacc.yacc()

res = parser.parse("parser_input") # the input
print(res)

ERROR: Bad precedence table


YaccError: Unable to build parser